# Data Quality Visualizations

In [ ]:
import time
import boto3
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import contextily as cx

# Set AWS credentials
s3 = boto3.resource("s3")
s3_cl = boto3.client("s3")  # for lower-level processes

# Set relative paths to other folders and objects in repository.
BUCKET_NAME = "wecc-historical-wx"
QAQC_DIR = "3_qaqc_wx"
MERGE_DIR = "4_merge_wx"
stations_csv_path = f"s3://{BUCKET_NAME}/{QAQC_DIR}/all_network_stationlist_qaqc.csv"
shapepath = "s3://wecc-historical-wx/0_maps/tl_2021_us_state"

In [ ]:
station_list = pd.read_csv(stations_csv_path)

In [ ]:
total_flag_path = f"s3://{BUCKET_NAME}/{MERGE_DIR}/total_flag_counts_native_timestep.csv"
total_flag_list = pd.read_csv(total_flag_path)

Append, onto station_list, target variable flag rate.
We want the flag rate per variable per station. Let's test with ASOSAWOS.

In [17]:
network_flag_path = f"s3://{BUCKET_NAME}/{MERGE_DIR}/ASOSAWOS/eraqc_counts_native_timestep/ASOSAWOS_A0705300346_flag_counts_native_timestep.csv"
flag_df_1_hello = pd.read_csv(network_flag_path)

In [23]:
network_flag_path_2 = f"s3://{BUCKET_NAME}/{MERGE_DIR}/ASOSAWOS/eraqc_counts_native_timestep/ASOSAWOS_A0685400115_flag_counts_native_timestep.csv"
flag_df_2_hello = pd.read_csv(network_flag_path_2)

In [27]:
flag_df_1_hello

,eraqc_flag_values,elevation,pr,ps_altimeter,ps,sfcWind_dir,sfcWind,tas,tdps
0,no_flag,140066,140066,140066,140066,140066,135752,140066,135949
1,27.0,0,0,0,0,0,4314,0,1691
2,28.0,0,0,0,0,0,0,0,1733
3,13.0,0,0,0,0,0,0,0,693
4,total_obs_count,140066,140066,140066,140066,140066,140066,140066,140066


In [28]:
flag_df_2_hello

,eraqc_flag_values,elevation,pr,ps_altimeter,ps,sfcWind_dir,sfcWind,tas,tdps
0,no_flag,205851,205851,205851,205851,205851,205851,205851,205387
1,27.0,0,0,0,0,0,0,0,464
2,total_obs_count,205851,205851,205851,205851,205851,205851,205851,205851


In [24]:
flag_df_1 = flag_df_1_hello.set_index("eraqc_flag_values")
subset = flag_df_1[~flag_df_1.index.isin(["no_flag", "total_obs_count"])]

totals = subset.sum(numeric_only=True)
flag_df_1.loc["total_flag"] = pd.Series(totals)

frac = flag_df_1.loc["total_flag"] / flag_df_1.loc["total_obs_count"]
flag_df_1.loc["frac"] = pd.Series(frac)

rates_df = flag_df_1.loc[["frac"]]
rates_df = rates_df.rename(index={"frac": 'station_name'})

In [34]:
flag_df_2 = flag_df_2_hello.set_index("eraqc_flag_values")
subset_2 = flag_df_2[~flag_df_2.index.isin(["no_flag", "total_obs_count"])]

totals = subset_2.sum(numeric_only=True)
flag_df_2.loc["total_flag"] = pd.Series(totals)

frac_2= flag_df_2.loc["total_flag"] / flag_df_2.loc["total_obs_count"]
flag_df_2.loc["frac"] = pd.Series(frac_2)

rates_df_2 = flag_df_2.loc[["frac"]]
rates_df_2 = rates_df_2.rename(index={"frac": "station_name_2"})

In [41]:
rates_df = rates_df.reset_index()

In [49]:
rates_df

,eraqc_flag_values,elevation,pr,ps_altimeter,ps,sfcWind_dir,sfcWind,tas,tdps
0,station_name,0.0,0.0,0.0,0.0,0.0,0.0308,0.0,0.029393


In [43]:
rates_df_2 = rates_df_2.reset_index()

In [50]:
rates_df_2

,eraqc_flag_values,elevation,pr,ps_altimeter,ps,sfcWind_dir,sfcWind,tas,tdps
0,station_name_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002254


In [53]:
test = pd.merge(rates_df,rates_df_2, how='outer')

In [54]:
test

,eraqc_flag_values,elevation,pr,ps_altimeter,ps,sfcWind_dir,sfcWind,tas,tdps
0,station_name,0.0,0.0,0.0,0.0,0.0,0.0308,0.0,0.029393
1,station_name_2,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.002254


In [64]:
network_flag_path_3 = f"s3://{BUCKET_NAME}/{MERGE_DIR}/ASOSAWOS/eraqc_counts_native_timestep/ASOSAWOS_99999994176_flag_counts_native_timestep.csv"
flag_df_3_hello = pd.read_csv(network_flag_path_3)

In [75]:
station_name = network_flag_path.split(f"s3://{BUCKET_NAME}/{MERGE_DIR}/ASOSAWOS/eraqc_counts_native_timestep" + "/")[1]

In [76]:
station_name

'ASOSAWOS_A0705300346_flag_counts_native_timestep.csv'

In [ ]:
def _pairwise_rate(flag_df_1: pd.DataFrame, flag_df_2: pd.DataFrame,station_name: str) -> pd.DataFrame:
    """
    Sums two input flag count dataframes. This is a helper function for sum_flag_counts().

    Parameters
    ----------
    flag_df_1: pd.DataFrame
        dataframe of previously summed station flag counts
    flag_df_2: pd.DataFrame
        flag counts dataframes for next station

    Returns
    -------
    summed_df: pd.DataFrame

    """
    if len(flag_df_2) == 0:
        return flag_df_1
    else:
        flag_df_1 = flag_df_1.set_index("eraqc_flag_values")
        subset = flag_df_1[~flag_df_1.index.isin(["no_flag", "total_obs_count"])]

        totals = subset.sum(numeric_only=True)
        flag_df_1.loc["total_flag"] = pd.Series(totals)

        frac = flag_df_1.loc["total_flag"] / flag_df_1.loc["total_obs_count"]
        flag_df_1.loc["frac"] = pd.Series(frac)

        rates_df = flag_df_1.loc[["frac"]]
        rates_df = rates_df.rename(index={"frac": station_name})

        # now append to flag_df_2
        rates_df = pd.merge(rates_df, flag_df_2, how="outer")

        return rates_df

In [72]:
test_1 = _pairwise_rate(flag_df_1_hello,flag_df_2_hello,'ASOAWOS')

In [73]:
test_1

,elevation,pr,ps_altimeter,ps,sfcWind_dir,sfcWind,tas,tdps
eraqc_flag_values,,,,,,,,
ASOAWOS,0.0,0.0,0.0,0.0,0.0,0.0308,0.0,0.029393


In [60]:
network = "VCAPCD"

In [61]:
# the function iteratively adds in flag counts to this dataframe
flag_rate_df = []

# point to folder containing station flag count CSVs
flags_prefix = f"{MERGE_DIR}/{network}/eraqc_counts_native_timestep"

## Merge flag counts

# loop through all CSVs are the given level
for item in s3.Bucket(BUCKET_NAME).objects.filter(Prefix=flags_prefix):
    obj = s3_cl.get_object(Bucket=BUCKET_NAME, Key=item.key)
    flags = pd.read_csv(obj["Body"])
    station_name = item.key.split(flags_prefix + "/")[1]
    # the CSV is empty
    if flags.empty:
        continue
    # the CSV is not empty
    else:
        # send current dataframe and dataframe of previously summed counts to helper function
        flag_rate_df = _pairwise_rate(flags, flag_rate_df,station_name)
        # print(station_name)

/tmp/ipykernel_1199/3068948477.py:33: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  rates_df = pd.merge(rates_df, flag_df_2, how="outer")


In [62]:
flag_rate_df

,elevation,hurs,pr_1h,rsds,sfcWind_dir,sfcWind,tas,tdps_derived,ps,pr,eraqc_flag_values
0,0.0,0.001498,0.000279,0.001347,0.002822,0.001486,0.003403,0.009395,NaN,NaN,NaN
1,0.0,0.001683,0.000000,0.000856,0.002733,0.001537,0.001780,0.007226,0.000311,NaN,NaN
2,0.0,0.002337,0.000000,0.002046,0.003850,0.002308,0.004024,0.009144,0.001435,0.0,NaN
3,0.0,0.001378,0.000000,0.001227,0.001644,0.001355,0.000706,0.006322,NaN,NaN,NaN
4,0.0,0.000908,0.000232,0.008078,0.001314,0.000908,0.000502,0.003614,NaN,0.0,NaN
5,103040.0,102868.000000,103040.000000,102976.000000,102394.000000,102961.000000,102806.000000,102332.000000,103035.000000,103040.0,no_flag
6,0.0,172.000000,0.000000,64.000000,79.000000,79.000000,91.000000,89.000000,2.000000,0.0,11.0
7,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,40.000000,3.000000,0.0,23.0
8,0.0,0.000000,0.000000,0.000000,567.000000,0.000000,0.000000,0.000000,0.000000,0.0,15.0
9,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,102.000000,451.000000,0.000000,0.0,26.0


## Map

In [ ]:
# Format dates in datetime format (this gets lost in import).
station_list["start-date"] = pd.to_datetime(station_list["start-date"], utc=True)
station_list["end-date"] = pd.to_datetime(station_list["end-date"], utc=True)

# Make a geodataframe.
gdf = gpd.GeoDataFrame(
    station_list,
    geometry=gpd.points_from_xy(station_list.longitude, station_list.latitude),
)
gdf.set_crs(epsg=4326, inplace=True)  # Set CRS

# Project data to match base tiles.
gdf_wm = gdf.to_crs(epsg=3857)  # Web mercator

# Read in geometry of continental US.
us = gpd.read_file(shapepath)

# Remove territories, AK, HI
rem_list = ["HI", "AK", "MP", "GU", "AS", "PR", "VI"]
us = us.loc[us.STUSPS.isin(rem_list) == False]

# Use to clip stations
us = us.to_crs(epsg=3857)
gdf_us = gdf_wm.clip(us)

# Plot
ax = gdf_us.plot(
    "network",
    figsize=(15, 15),
    alpha=1,
    markersize=3,
    legend=True,
    cmap="nipy_spectral",
)
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron)
ax.set_axis_off()

